In [48]:
import requests
import bs4
import lxml
import re
from tqdm import tqdm as tq
from time import sleep
from random import randint
from datetime import datetime
import pandas as pd
import pathlib
import urllib
import json

In [2]:
url=[]
history_url=[]
filter_urls=[]
count=0
for i in tq(range(1,401)):
    results=requests.get('https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count')
    soup=bs4.BeautifulSoup(results.text)
    urls=[item.get('href') for item in soup.find_all('a') if item.get('class')== ['content-card','content-card-place'] ]
    count += len(urls)
    history_url=history_url+urls


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [06:00<00:00,  1.11it/s]


In [14]:
x=pd.DataFrame({'A':history_url})
x.to_csv('url.csv')

In [287]:
history_url

['/places/city-hall-station',
 '/places/highgate-cemetery',
 '/places/leadenhall-market',
 '/places/wave-organ',
 '/places/catacombes-de-paris',
 '/places/evolution-store',
 '/places/magowans-infinite-mirror-maze',
 '/places/underground-tunnels-los-angeles',
 '/places/secret-tiled-staircase',
 '/places/dream-house',
 '/places/the-museum-of-death-los-angeles-california',
 '/places/last-bookstore',
 '/places/gods-own-junkyard',
 '/places/echo-park-time-travel-mart',
 '/places/the-ruins-of-st-dunstan-in-the-east-london-england',
 '/places/fingal-s-cave',
 '/places/wave-rock',
 '/places/winchester-mystery-house',
 '/places/roosevelt-island-smallpox-hospital-ruins',
 '/places/eastern-state-penitentiary',
 '/places/mutter-museum',
 '/places/fly-ranch-geyser',
 '/places/antelope-canyon',
 '/places/the-elevated-acre-new-york-new-york',
 '/places/thor-s-well',
 '/places/mmuseumm-2',
 '/places/seward-street-slides',
 '/places/ghibli-museum',
 '/places/spreepark',
 '/places/twinings-tea-shop',
 '

In [31]:
def store_html(results,i,file):

    new_dir=pathlib.Path('D:/Anaconda/my-work/ADM/','place_html_1/place'+str(i))
    new_dir.mkdir(parents=True, exist_ok=True)
    # You have to make a file inside the new directory
    y=file+'.html'
    new_file = new_dir / y
    f=open(new_file,'wb')
    
    f.write(results.content)
    f.close()

In [341]:
def save_dic(dic,i,file):
    new_dir=pathlib.Path('D:/Anaconda/my-work/ADM/','result'+str(i))
    new_dir.mkdir(parents=True, exist_ok=True)
    # You have to make a file inside the new directory
    y=file+'.txt'
    new_file = new_dir / y
    dic={'Dict':dic}
    f=open(new_file,'w')
    
    f.write(json.dumps(dic))
    f.close()

In [330]:
z=re.findall('\/\w+\/(.*)','/places/museum-vampires')


['museum-vampires']


In [6]:
def extract_single_place(url,index,file_name):
    results=requests.get(url)
    store_html(results,index,file_name)
    soup=bs4.BeautifulSoup(results.text)
    place_information={}
    ###Find Title###
    try:
        for item in soup.find_all('h1'): 
            if item.get('class')==['DDPage__header-title']:

                    place_information={'placeName':str(item.get_text(strip=True,separator=' '))}
    except:
                place_information={'placeName':''}
    ### Find Location(Tags):
    try:
        Tag =soup.find('div',{'class':"DDPage__header-place-location"}).get_text(strip=True,separator=' ')
        place_information['placeTags']=Tag
    except:
        place_information['placeTags']=''
    ### Find Number of people visited
    try:
        number_of_people_visited=soup.find('div',{'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'}).get_text(strip=True,separator=' ')
        place_information['numPeopleVisited']=int(re.search('\d+',number_of_people_visited).group())
    except:
        place_information['numPeopleVisited']=''
    
    ### Find Number of People who want to go
    try:
        people_want_go=soup.find('div',{'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'}).get_text(strip=True,separator=' ')
        place_information['numPeopleWant']=int(re.search('\d+',people_want_go).group())
    except:
        place_information['numPeopleWant']=''
        
    #### Place description
    try:
        places_des=soup.find('div',{'class':['DDP__body-copy']}).get_text(strip=True,separator=' ')
        place_information['placeDesc']=str(places_des)
    except:
        place_information['placeDesc']=''
    ### Short description:
    try:
        title1=soup.find('h1',{'class':['DDPage__header-title']}).get_text(strip=True,separator=' ')
        Tag =soup.find('div',{'class':"DDPage__header-place-location"}).get_text(strip=True,separator=' ')
        short_des=soup.find('h3',{'class':['DDPage__header-dek']}).get_text(strip=True,separator=' ')
        place_information['placeShortDesc']=str(title1)+','+str(Tag)+', '+str(short_des) ###Seperate by commas
    except:
        place_information['placeShortDesc']=''
    
    ### Nearby places
    try:
        places=[]
        for items in soup.find_all('div',{'class':['DDPageSiderailRecirc__item-title']}):
            places.append(items.get_text(strip=True,separator=' '))
        place_information['placeNearby']=list(set(places))
    except:
        place_information['placeNearby']=''
        
    ### Place Address:
    try:
        items=soup.find('address',{'class':['DDPageSiderail__address']})
        x=items.find('div',attrs={'class':None}) 
        ## Check the example for some idea of it's functioning
        y=x.get_text(strip=True,separator='')
        z=x.find('div').get_text(strip=True,separator='')
        place_information['placeAddress']=y.rsplit(z)[0]
    except:
        place_information['placeAddress']=''    
    ### Place Altitude and Latitude
    try:
        items=soup.find('div',{'class':['DDPageSiderail__coordinates js-copy-coordinates']})
        cord=items.get_text(strip=True,separator=' ').rsplit(',')
        place_information['placeAlt']=float(cord[0])   #### FLoat or integers????
        place_information['placeLong']=float(cord[1])   
    except:
        place_information['placeAlt']=''  #### FLoat or integers????
        place_information['placeLong']=''   
    
    #### Post editors:
    try:
        editors=[]
        for items in soup.find_all('li',{'class':["DDPContributorsList__item"]}):
            editors.append(items.find('span').get_text(strip=True,separator=' '))

        place_information['placeEditors']=editors
    except:
        place_information['placeEditors']=''
            
    ###Post publishing date
    try:
        items=soup.find('div',{'class':["DDPContributor__name"]})
        date=items.get_text(strip=True,separator='')
        #date_time_obj = datetime.strptime(date, '%b %d, %Y')
        place_information['placePubDate']=date     
    except:
        place_information['placePubDate']=''
    ### Finding related places and Appeared list
    try:
        var=[]
        for items in soup.find_all('div',{'class':["card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"]}):
            var.append(items.find_all('h3',{'class':["Card__heading --content-card-v2-title js-title-content"]}))
        Related_places=[]
        for j in var[1]:
                Related_places.append(j.find('span').get_text(strip=True,separator=' '))
        Appeared_list=[]
        for j in var[2]:
                Appeared_list.append(j.find('span').get_text(strip=True,separator=' '))
        place_information['placeRelatedLists']=Appeared_list
        place_information['placeRelatedPlaces']=Related_places
        place_information['placeURL']=url
    except:
        place_information['placeRelatedLists']=''
        place_information['placeRelatedPlaces']=''
        place_information['placeURL']=''

#     save_dic(place_information,index,file_name)
                    
    return place_information           
              

In [347]:
import csv
x=extract_single_place('https://www.atlasobscura.com'+history_url[0])
with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in x.items():
       writer.writerow([key, value])

TypeError: extract_single_place() missing 2 required positional arguments: 'index' and 'file_name'

In [18]:
list_dict=[]
for i,j in tq(enumerate(history_url)):
 index=(i)//18 + 1
 z=re.findall('\/\w+\/(.*)',history_url[i])

 sleep(randint(1,3))
 list_dict.append(extract_single_place('https://www.atlasobscura.com'+history_url[i],index,z[0]))




7200it [6:02:06,  3.02s/it]


In [39]:
list_dict=[]
index=(3387)//18 + 1
z=re.findall('\/\w+\/(.*)',history_url[3387])
list_dict.append(extract_single_place('https://www.atlasobscura.com'+history_url[3387],index,z[0]))

In [40]:

for i,j in tq(enumerate(list_dict)):
    for k in j.keys():
        x.at[3387,k]=j[k]


1it [00:00, 1015.08it/s]


In [36]:
x=pd.read_csv('final_dataset.csv')

In [46]:
x.to_csv('final_dataset_1.csv')

In [403]:
z=re.findall('\/\w+\/(.*)',history_url[0])
extract_single_place('https://www.atlasobscura.com'+history_url[0],index,z[0])

{'placeName': 'City Hall Station',
 'placeTags': 'Manhattan, New York',
 'numPeopleVisited': 1824,
 'numPeopleWant': 8568,
 'placeDesc': 'The first New York City subway was built and operated by the Interborough Rapid Transit Company (IRT) and opened on October 27, 1904, to the joy of New York elevated train and streetcar riders. The City Hall station on the IRT local track was lavished with fine architectural details, including glass tiles and large chandeliers. However, the Gustavino vaulted ceilings and skylights were lost on busy commuters, and the stop was one of the least-used in the system. It was the only station that did not have turnstiles installed by 1923, and the nearby Brooklyn Bridge stop was frequented by the express train and closer to connecting streetcars. Because of the curved platform, cars with center doors could not be used at this station unless they had specially modified door controls which allowed just the end doors to be opened. In 1945, the station was clos

In [30]:
history_url[3387]

'/places/la-posada-del-sol'

In [29]:
x[x.placeName.isna()]

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
3387,NaN,,,,,,[],,,,[],,,,


In [47]:
x

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"Manhattan, New York",1824.0,8569.0,The first New York City subway was built and o...,"City Hall Station,Manhattan, New York, A beaut...","['Tunnel Number 3', 'African Burial Ground Nat...","31 Centre StNew York, New York, 10007United St...",40.7134,-74.0046,"['charding407', 'fosterc827', 'Mike H', 'erjef...","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"London, England",2615.0,8179.0,"Opened in 1839, Highgate is one of London’s mo...","Highgate Cemetery,London, England, London's cr...","['Dick Whittington’s Cat', ""World's Largest Po...","Swain's Lane, HighgateLondon, England, N6Unite...",51.5675,-0.1483,"['thesecretcity', 'thedyerghoulhouse', 'juliap...","August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"London, England",3133.0,7560.0,The ornate 19th-century painted roof and cobbl...,"Leadenhall Market,London, England, This ornate...","['The Cornhill Devils', ""London's Original and...","London, England, EC3VUnited Kingdom",51.5126,-0.0834,"['Julius Spada', 'Katecrimson', 'thesecretcity...","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"San Francisco, California",2426.0,7400.0,"Located on a jetty in the San Francisco Bay, t...","The Wave Organ,San Francisco, California, A hu...","['The Stern of the Brigantine Galilee', 'Long ...","83 Marina Green DrSan Francisco, California, 9...",37.8085,-122.4401,"['routt1111', 'wefiddleboy14', 'Saal333', 'Nei...","November 21, 2008","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"Paris, France",4442.0,7036.0,"In 2004, Parisian police were assigned to do a...","Catacombes de Paris,Paris, France, The vast, l...","['Sculptures de Décure', ""Jeannot's Floor"", 'A...","1 Place Denfert-RochereauParis, 75014France",48.8343,2.3322,"['marypippen', 'ramonrodz2212', 'mathewbrowne'...","February 13, 2009","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",https://www.atlasobscura.com/places/catacombes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,Xul Solar Museum,"Buenos Aires, Argentina",129.0,405.0,With dreams of reforming and perfecting the un...,"Xul Solar Museum,Buenos Aires, Argentina, Xul ...","['Ricardo Rojas House Museum', 'Villa Freud', ...",Laprida 1212Buenos AiresArgentina,-34.5948,-58.4078,[],"September 12, 2016","[""A Linguistics Lover's Tour of the World""]","['Blenko Man', 'Ilana Goor Museum', 'Museo de ...",https://www.atlasobscura.com/places/xul-solar-...
7196,Klatovy Catacomb Mummies,"Klatovy, Czechia",49.0,405.0,In an underground crypt beneath a Jesuit churc...,"Klatovy Catacomb Mummies,Klatovy, Czechia, The...","['Drachenstich-Festspiele', 'Poledník Lookout ...",DenisovaKlatovyCzechia,49.3951,13.2918,[],"February 25, 2016","['19 Catacombs Sure to Tingle Your Spine', ""Bo...","['Rothwell Bone Crypt', 'Ossario di San Martin...",https://www.atlasobscura.com/places/klatovy-ca...
7197,Termesphere Gallery,"Spearfish, South Dakota",71.0,405.0,"Most artists paint on a a flat plane, but in t...","Termesphere Gallery,Spearfish, South Dakota, T...","['Fish Car No. 3', 'Mount Roosevelt Friendship...","1920 Christensen DriveSpearfish, South Dakota,...",44.4574,-103.8258,[],"August 19, 2014",NaN,NaN,NaN
7198,Wheeler Peak,"Taos Ski Valley, New Mexico",241.0,405.0